In [1]:
import sys
import time
from wideboost.wrappers import wxgb
import numpy as np
import xgboost as xgb

import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=False,
    as_supervised=True,
    with_info=True,
)

for i in ds_train.batch(60000):
    a = i
    
for i in ds_test.batch(60000):
    b = i

In [2]:
xtrain = a[0].numpy().reshape([-1,28*28])
ytrain = a[1].numpy()

xtest = b[0].numpy().reshape([-1,28*28])
ytest = b[1].numpy()

In [3]:
def _onehot(Y):
    b = np.zeros((Y.shape[0], Y.max().astype(int)+1))
    b[np.arange(Y.shape[0]),Y.astype(int).flatten()] = 1
    return b

In [4]:
def get_data(seed):
    np.random.seed(seed)
    train_idx = np.random.choice(range(xtrain.shape[0]), round(xtrain.shape[0] * 0.7), replace=False)
    valid_idx = np.array(list(set(range(xtrain.shape[0])) - set(train_idx)))
    train_idx = np.random.choice(train_idx, 3_000, replace=False)
    valid_idx = np.random.choice(valid_idx, 2_000, replace=False)

    test_idx = np.random.choice(range(xtest.shape[0]), 1_000, replace=False)

    features = np.random.choice(range(xtrain.shape[1]), 200, replace=False)


    y_train = _onehot(ytrain[train_idx])
    y_valid = _onehot(ytrain[valid_idx])
    y_test =  _onehot(ytest[test_idx])

    return (
        xtrain[train_idx[:,None], features],
        xtrain[valid_idx[:,None], features],
        xtest[test_idx[:,None], features], y_train, y_valid, y_test
    )

In [6]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval
import os

iters = 30
results = []
for est in range(iters):

    t0 = time.time()
    x_train, x_valid, x_test, y_train, y_valid, y_test = get_data(1000 + est)
    watchlist = [
        ((x_train, y_train),'train'),
        ((x_test, y_test), 'test'),
        ((x_valid, y_valid), 'valid')
    ]

    best_val = 1000.0
    xgb_test_val = 1000.0
    xgb_param = {}
    nrounds = 0

    def objective(param):
        global best_val
        global xgb_test_val
        global xgb_param
        global nrounds
        num_round = 500
        param['output_dim'] = 10
        param['nthread'] = 2
        print(param)

        xgb_results = {}
        wbst = wxgb.fit(
            x_train, y_train,
            param, num_round, watchlist,
            evals_result=xgb_results,
            early_stopping_rounds=20,
            verbose_eval=200
        )
        
        if (min(xgb_results['valid']['many_logloss'])) < best_val:
            print("NEW BEST VALUE!")
            best_val = min(xgb_results['valid']['many_logloss'])
            xgb_test_val = xgb_results['test']['many_logloss'][
                np.argmin(xgb_results['valid']['many_logloss'])
            ]
            xgb_param = param
            nrounds = np.argmin(xgb_results['valid']['many_logloss']) + 1

        return {'loss': min(xgb_results['valid']['many_logloss']), 'status': STATUS_OK }

    spc = {
        'btype': hp.choice('btype',['I']),
        'extra_dims': hp.choice('extra_dims',[0]),
        'objective': hp.choice('objective',['manybinary:logistic']),
        'eval_metric':hp.choice('eval_metric',[['many_logloss']]),
        'eta': hp.loguniform('eta', -7, 0),
        'beta_eta': hp.choice('beta_eta',[0]),
        'max_depth' : hp.choice('max_depth',range(1,11)),
        'subsample': hp.uniform('subsample', 0.5, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
        'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
        'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
        'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
        'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
        'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
    }
    os.environ['HYPEROPT_FMIN_SEED'] = '101010'
    best = fmin(objective,
        space=spc,
        algo=tpe.suggest,
        max_evals=25)
    t1 = time.time()
    results.append({
        'iter': est,
        'valid_logloss': best_val,
        'test_logloss': xgb_test_val,
        'num_rounds': nrounds,
        'time_diff': t1 - t0,
        'winning_param': xgb_param.copy()
    })

{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.7489980469009814, 'colsample_bytree': 0.9420146676089547, 'eta': 0.0070706936561268224, 'eval_metric': ('many_logloss',), 'extra_dims': 0, 'gamma': 0, 'lambda': 0, 'max_depth': 5, 'min_child_weight': 2.1041667775099838e-05, 'objective': 'manybinary:logistic', 'subsample': 0.6262451727083667, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.68707	test-many_logloss:0.68727	valid-many_logloss:0.68724
[200]	train-many_logloss:0.18431	test-many_logloss:0.20818	valid-many_logloss:0.20662
[400]	train-many_logloss:0.07420	test-many_logloss:0.11057	valid-many_logloss:0.10850
[499]	train-many_logloss:0.05209	test-many_logloss:0.09254	valid-many_logloss:0.09044
NEW BEST VALUE!                                       
{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.8357246683355697, 'colsample_bytree': 0.7491187395188089, 'eta': 0.605518780660455, 'eval_metric': ('many_logloss',), 'extra_dims': 0, 'gamma': 0, '

../wideboost/wideboost/evals/classification.py:82: RuntimeWarning: invalid value encountered in log
  return -np.mean(np.log(p))

../wideboost/wideboost/evals/classification.py:82: RuntimeWarning: divide by zero encountered in log
  return -np.mean(np.log(p))

../wideboost/wideboost/evals/classification.py:80: RuntimeWarning: overflow encountered in exp
  p = 1/(1 + np.exp(-logits))

../wideboost/wideboost/objectives/manybinarylogloss.py:14: RuntimeWarning: overflow encountered in exp
  P = 1 / (1 + np.exp(-logits))



[23]	train-many_logloss:nan	test-many_logloss:nan	valid-many_logloss:nan
{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.7061150549075759, 'colsample_bytree': 0.5843009197915733, 'eta': 0.002630107568123181, 'eval_metric': ('many_logloss',), 'extra_dims': 0, 'gamma': 6.947115616562489, 'lambda': 0, 'max_depth': 10, 'min_child_weight': 41.36978142254861, 'objective': 'manybinary:logistic', 'subsample': 0.7453409993563878, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.69120	test-many_logloss:0.69121	valid-many_logloss:0.69122
[200]	train-many_logloss:0.43210	test-many_logloss:0.43422	valid-many_logloss:0.43461
[400]	train-many_logloss:0.30839	test-many_logloss:0.31178	valid-many_logloss:0.31227
[499]	train-many_logloss:0.27036	test-many_logloss:0.27420	valid-many_logloss:0.27475
{'alpha': 1.055992173502114e-05, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.8017715093779021, 'colsample_bytree': 0.7625111741024486, 'eta': 0.007606220882567551, 'eval_metric

[0]	train-many_logloss:0.43751	test-many_logloss:0.43986	valid-many_logloss:0.44247
[123]	train-many_logloss:0.05417	test-many_logloss:0.09020	valid-many_logloss:0.08868
{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.8025654416649992, 'colsample_bytree': 0.8748052006921443, 'eta': 0.009393489875683971, 'eval_metric': ('many_logloss',), 'extra_dims': 0, 'gamma': 0, 'lambda': 0, 'max_depth': 2, 'min_child_weight': 0.013234942416650719, 'objective': 'manybinary:logistic', 'subsample': 0.8694541664166509, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.68619	test-many_logloss:0.68622	valid-many_logloss:0.68622
[200]	train-many_logloss:0.21970	test-many_logloss:0.22572	valid-many_logloss:0.22511
[400]	train-many_logloss:0.14107	test-many_logloss:0.15176	valid-many_logloss:0.15049
[499]	train-many_logloss:0.12216	test-many_logloss:0.13483	valid-many_logloss:0.13362
{'alpha': 3.1952805148613077, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.8947364949908566, '

[0]	train-many_logloss:0.69241	test-many_logloss:0.69242	valid-many_logloss:0.69242
[200]	train-many_logloss:0.56880	test-many_logloss:0.56994	valid-many_logloss:0.56973
[400]	train-many_logloss:0.47978	test-many_logloss:0.48179	valid-many_logloss:0.48157
[499]	train-many_logloss:0.44448	test-many_logloss:0.44692	valid-many_logloss:0.44671
{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.787125082065971, 'colsample_bytree': 0.5243474561426513, 'eta': 0.8099003407203538, 'eval_metric': ('many_logloss',), 'extra_dims': 0, 'gamma': 7.475715249737038e-05, 'lambda': 0.0003639646509775927, 'max_depth': 7, 'min_child_weight': 0.09759627970810976, 'objective': 'manybinary:logistic', 'subsample': 0.553442319017114, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.24943	test-many_logloss:0.28692	valid-many_logloss:0.28708
[22]	train-many_logloss:nan	test-many_logloss:nan	valid-many_logloss:nan
{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.70611505490757

...

[0]	train-many_logloss:0.66927	test-many_logloss:0.66962	valid-many_logloss:0.66964
[200]	train-many_logloss:0.05695	test-many_logloss:0.08467	valid-many_logloss:0.08715
[400]	train-many_logloss:0.02295	test-many_logloss:0.06536	valid-many_logloss:0.06571
[499]	train-many_logloss:0.01566	test-many_logloss:0.06299	valid-many_logloss:0.06241
NEW BEST VALUE!                                                        
{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.7211698432094364, 'colsample_bytree': 0.712394975511369, 'eta': 0.008477424619483462, 'eval_metric': ('many_logloss',), 'extra_dims': 0, 'gamma': 0, 'lambda': 0, 'max_depth': 3, 'min_child_weight': 32.756265617720345, 'objective': 'manybinary:logistic', 'subsample': 0.6995489417449295, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.68682	test-many_logloss:0.68687	valid-many_logloss:0.68689
[200]	train-many_logloss:0.21840	test-many_logloss:0.22471	valid-many_logloss:0.22798
[400]	train-many_logloss:0.13743

[400]	train-many_logloss:0.00561	test-many_logloss:0.06751	valid-many_logloss:0.06568
[470]	train-many_logloss:0.00333	test-many_logloss:0.06800	valid-many_logloss:0.06556
NEW BEST VALUE!                                                       
{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.6189065258301154, 'colsample_bytree': 0.7882001782331782, 'eta': 0.003770077706590942, 'eval_metric': ('many_logloss',), 'extra_dims': 0, 'gamma': 0.0006381725365621713, 'lambda': 0, 'max_depth': 1, 'min_child_weight': 0.05537987461653371, 'objective': 'manybinary:logistic', 'subsample': 0.6537419349296045, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.69057	test-many_logloss:0.69057	valid-many_logloss:0.69057
[200]	train-many_logloss:0.40183	test-many_logloss:0.40355	valid-many_logloss:0.40340
[400]	train-many_logloss:0.30077	test-many_logloss:0.30320	valid-many_logloss:0.30350
[499]	train-many_logloss:0.27398	test-many_logloss:0.27668	valid-many_logloss:0.27716
{'alpha':

[0]	train-many_logloss:0.63591	test-many_logloss:0.63732	valid-many_logloss:0.63714
[200]	train-many_logloss:0.02169	test-many_logloss:0.06736	valid-many_logloss:0.06583
[400]	train-many_logloss:0.01604	test-many_logloss:0.06520	valid-many_logloss:0.06335
[499]	train-many_logloss:0.01527	test-many_logloss:0.06496	valid-many_logloss:0.06312
NEW BEST VALUE!                                                        
{'alpha': 1.200782835459218, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.8687918622527812, 'colsample_bytree': 0.6034148520201185, 'eta': 0.19484734134839077, 'eval_metric': ('many_logloss',), 'extra_dims': 0, 'gamma': 0, 'lambda': 1.635393836955409e-05, 'max_depth': 4, 'min_child_weight': 0.0005216471023949306, 'objective': 'manybinary:logistic', 'subsample': 0.7985877624194806, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.54837	test-many_logloss:0.55120	valid-many_logloss:0.55157
[139]	train-many_logloss:0.00663	test-many_logloss:0.06298	valid-many_logloss:0

[0]	train-many_logloss:0.68792	test-many_logloss:0.68793	valid-many_logloss:0.68795
[200]	train-many_logloss:0.29484	test-many_logloss:0.29578	valid-many_logloss:0.29790
[400]	train-many_logloss:0.21848	test-many_logloss:0.22030	valid-many_logloss:0.22325
[499]	train-many_logloss:0.19801	test-many_logloss:0.20041	valid-many_logloss:0.20344
{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.98982155285091, 'colsample_bytree': 0.8137063678634311, 'eta': 0.021437825449384364, 'eval_metric': ('many_logloss',), 'extra_dims': 0, 'gamma': 0, 'lambda': 0, 'max_depth': 5, 'min_child_weight': 2.5600171535202e-07, 'objective': 'manybinary:logistic', 'subsample': 0.7174114308249333, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.67468	test-many_logloss:0.67528	valid-many_logloss:0.67542
[200]	train-many_logloss:0.03248	test-many_logloss:0.07832	valid-many_logloss:0.07956
[400]	train-many_logloss:0.00660	test-many_logloss:0.06477	valid-many_logloss:0.06455
[498]	train-many_l

{'alpha': 3.1952805148613077, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.8947364949908566, 'colsample_bytree': 0.6489201195603208, 'eta': 0.04857283281713467, 'eval_metric': ('many_logloss',), 'extra_dims': 0, 'gamma': 0, 'lambda': 5.599926914440914e-07, 'max_depth': 4, 'min_child_weight': 1.7794291782520545e-07, 'objective': 'manybinary:logistic', 'subsample': 0.5893278373512121, 'output_dim': 10, 'nthread': 2}
[0]	train-many_logloss:0.65547	test-many_logloss:0.65585	valid-many_logloss:0.65615
[200]	train-many_logloss:0.03912	test-many_logloss:0.07298	valid-many_logloss:0.07270
[400]	train-many_logloss:0.02356	test-many_logloss:0.06506	valid-many_logloss:0.06420
[499]	train-many_logloss:0.02098	test-many_logloss:0.06416	valid-many_logloss:0.06324
NEW BEST VALUE!                                                        
{'alpha': 3.002743880356697, 'beta_eta': 0, 'btype': 'I', 'colsample_bylevel': 0.880750740505001, 'colsample_bytree': 0.6378931895802679, 'eta': 0.07394609656696

In [68]:
import pandas as pd
pd.DataFrame(results)

,iter,valid_logloss,test_logloss,num_rounds,time_diff,winning_param
0,0,0.060741,0.061365,500,1184.938146,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col..."
1,1,0.062533,0.054095,110,1097.437254,"{'alpha': 0.1617477004218055, 'beta_eta': 0, '..."
2,2,0.057733,0.056778,233,1135.469312,"{'alpha': 0.23062196782499855, 'beta_eta': 0, ..."
3,3,0.060273,0.058737,106,1092.539088,"{'alpha': 1.200782835459218, 'beta_eta': 0, 'b..."
4,4,0.057297,0.050025,216,1126.722315,"{'alpha': 0.7524380514093831, 'beta_eta': 0, '..."
5,5,0.061852,0.056350,119,1109.750677,"{'alpha': 0.19028592334432648, 'beta_eta': 0, ..."
6,6,0.057963,0.053345,157,1137.014389,"{'alpha': 0.19028592334432648, 'beta_eta': 0, ..."
7,7,0.061134,0.060308,125,1100.057296,"{'alpha': 1.200782835459218, 'beta_eta': 0, 'b..."
8,8,0.056773,0.053620,141,1208.929127,"{'alpha': 0.19028592334432648, 'beta_eta': 0, ..."
9,9,0.059144,0.059980,208,1104.502578,"{'alpha': 0.7524380514093831, 'beta_eta': 0, '..."
